Work in Progress

**Installing** dependency

we are keeping for now all the file of ALBERT in transformers.jl (it will Ultimately be added in TextAnalysis.jl with Transformers.jl as its dependency) we are using tokenize and Wordpiece(sentencepiece) from Albert which is based on Transformers.jl

In [49]:
using Transformers

Here we are taking 3 sentence as sample

In [50]:
sample1 = "God is Great! I won a lottery."
sample2 = "If all their conversations in the three months he had been coming to the diner were put together, it was doubtful that they would make a respectable paragraph."
sample3 = "She had the job she had planned for the last three years."
sample = [sample1,sample2,sample3]


3-element Array{String,1}:
 "God is Great! I won a lottery."                                                                                                                                 
 "If all their conversations in the three months he had been coming to the diner were put together, it was doubtful that they would make a respectable paragraph."
 "She had the job she had planned for the last three years."                                                                                                      

ALBERT uses SentencePiece for word segmentation and Sentence segementation.
Here we are going to provide both Sentencepiece as well as Wordpiece (using Vocab file of ALBERT) for ALBERT with the former as defalut one.
We first demonstrate WordPiece
To use word piece we need to make following changes as below


we first replace " "(space) with ' _' [space(U+2581)]

To decode split sentence. SentencePiece replace space with U+2581 and Store word in .vocab file with U+2581 as prefix .To tackle this we are using little trick as below.

In [51]:
We will replace it with better implementation of SentencePiece 

LoadError: syntax: extra token "will" after end of expression

In [52]:
sample[1] = replace(sample[1]," "=> " ▁")
sample[2] = replace(sample[2]," "=> " ▁")
sample[3] = replace(sample[3]," "=> " ▁")


"She ▁had ▁the ▁job ▁she ▁had ▁planned ▁for ▁the ▁last ▁three ▁years."

We Can use WordPiece for Word Segmentation with ALBERT vocabulary as follow. 
working on the APIs for Wordpiece

In [53]:
sample

3-element Array{String,1}:
 "God ▁is ▁Great! ▁I ▁won ▁a ▁lottery."                                                                                                                                                      
 "If ▁all ▁their ▁conversations ▁in ▁the ▁three ▁months ▁he ▁had ▁been ▁coming ▁to ▁the ▁diner ▁were ▁put ▁together, ▁it ▁was ▁doubtful ▁that ▁they ▁would ▁make ▁a ▁respectable ▁paragraph."
 "She ▁had ▁the ▁job ▁she ▁had ▁planned ▁for ▁the ▁last ▁three ▁years."                                                                                                                      

In [54]:
using Transformers.BidirectionalEncoder

In [55]:
tokens = tokenise.(sample, Val(true))


3-element Array{Array{String,1},1}:
 ["god", "▁is", "▁great", "!", "▁i", "▁won", "▁a", "▁lottery", "."]                                                                                                                                 
 ["if", "▁all", "▁their", "▁conversations", "▁in", "▁the", "▁three", "▁months", "▁he", "▁had"  …  "▁was", "▁doubtful", "▁that", "▁they", "▁would", "▁make", "▁a", "▁respectable", "▁paragraph", "."]
 ["she", "▁had", "▁the", "▁job", "▁she", "▁had", "▁planned", "▁for", "▁the", "▁last", "▁three", "▁years", "."]                                                                                      

In [56]:
vocab = readlines("/home/iamtejas/Downloads/albert_xlarge_v1/albert_xlarge/30k-clean.vocab")
vocabnew = split.(vocab , "\t")
vo = [vocabnew[1][1]]


1-element Array{SubString{String},1}:
 "<pad>"

We don't need Score of SentencePiece unigram model for Wordpiece.

In [57]:
for i in 1:30000
    vocab1 = vocabnew[i][1]
    push!(vo,vocab1)
end
vocab1 = convert(Array{String,1},vo)

30001-element Array{String,1}:
 "<pad>"        
 "<pad>"        
 "<unk>"        
 "[CLS]"        
 "[SEP]"        
 "[MASK]"       
 "("            
 ")"            
 "\""           
 "-"            
 "."            
 "–"            
 "£"            
 ⋮              
 "▁predation"   
 "▁saviour"     
 "▁archivist"   
 "▁obverse"     
 "error"        
 "▁tyrion"      
 "▁addictive"   
 "▁veneto"      
 "▁colloquial"  
 "agog"         
 "▁deficiencies"
 "▁eloquent"    

In [58]:
vocab1 = vocab1[2:30001]

30000-element Array{String,1}:
 "<pad>"        
 "<unk>"        
 "[CLS]"        
 "[SEP]"        
 "[MASK]"       
 "("            
 ")"            
 "\""           
 "-"            
 "."            
 "–"            
 "£"            
 "€"            
 ⋮              
 "▁predation"   
 "▁saviour"     
 "▁archivist"   
 "▁obverse"     
 "error"        
 "▁tyrion"      
 "▁addictive"   
 "▁veneto"      
 "▁colloquial"  
 "agog"         
 "▁deficiencies"
 "▁eloquent"    

In [59]:
wp = WordPiece(vocab1,"<unk>"; max_char = 200)

WordPiece(vocab_size=30000, unk=<unk>, max_char=200)

In [60]:
tokensnew = wp.(tokens)

3-element Array{Array{String,1},1}:
 ["god", "▁is", "▁great", "!", "▁i", "▁won", "▁a", "▁lottery", "."]                                                                                                                                 
 ["if", "▁all", "▁their", "▁conversations", "▁in", "▁the", "▁three", "▁months", "▁he", "▁had"  …  "▁was", "▁doubtful", "▁that", "▁they", "▁would", "▁make", "▁a", "▁respectable", "▁paragraph", "."]
 ["she", "▁had", "▁the", "▁job", "▁she", "▁had", "▁planned", "▁for", "▁the", "▁last", "▁three", "▁years", "."]                                                                                      

In [61]:
sample

3-element Array{String,1}:
 "God ▁is ▁Great! ▁I ▁won ▁a ▁lottery."                                                                                                                                                      
 "If ▁all ▁their ▁conversations ▁in ▁the ▁three ▁months ▁he ▁had ▁been ▁coming ▁to ▁the ▁diner ▁were ▁put ▁together, ▁it ▁was ▁doubtful ▁that ▁they ▁would ▁make ▁a ▁respectable ▁paragraph."
 "She ▁had ▁the ▁job ▁she ▁had ▁planned ▁for ▁the ▁last ▁three ▁years."                                                                                                                      

In [62]:
vocabalbert = Transformers.Basic.Vocabulary(wp.vocab,wp.vocab[wp.unk_idx])

Vocabulary(30000, unk=<unk>)

here we have indices form sample tokens.


In [63]:
indices = vocabalbert(tokensnew)

30×3 Array{Int64,2}:
  5475    822  1080
    26     66    42
   375     67    15
   188  13528  1206
    32     20    40
   231     15    42
    22    133  2036
 17519    819    27
    10     25    15
     2     42   237
     2     75   133
     2    881   123
     2     21    10
     ⋮             
     2     16     2
     2     33     2
     2     24     2
     2  22569     2
     2     31     2
     2     60     2
     2     84     2
     2    234     2
     2     22     2
     2  22740     2
     2  20600     2
     2     10     2

In [64]:
size(indices)

(30, 3)

since we only take one sentence as input

In [74]:
seg_indices = ones(Int, size(indices)...)

30×3 Array{Int64,2}:
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 ⋮      
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1

Next we need to turn those indices into embeddings.These can be done by loading embedding in Embed 

## SENTENCEPIECE
now i will show how to use Sentencepiece  
I am working on julia implementation of SentencePiece but for now we can use PyCall and also python Wrapper is under development


### Environment setup
You can install Python binary package of SentencePiece with.

> % pip install sentencepiece

For detail visit - [Sentencpiece](https://github.com/google/sentencepiece)


In [75]:
using PyCall
SentencePiece = pyimport("sentencepiece")
sp = SentencePiece.SentencePieceProcessor()
sp.Load("/home/iamtejas/Downloads/albert_xlarge_v1/albert_xlarge/30k-clean.model")


true

#### Avaliable APIs
> sp.EncodeAsPieces

> sp.EncodeAsIds

In [76]:
sample4 = "God is Great! I won a lottery."
sample5 = "If all their conversations in the three months he had been coming to the diner were put together, it was doubtful that they would make a respectable paragraph."
sample6 = "She had the job she had planned for the last three years."
sampleforsp = [sample4,sample5,sample6]

3-element Array{String,1}:
 "God is Great! I won a lottery."                                                                                                                                 
 "If all their conversations in the three months he had been coming to the diner were put together, it was doubtful that they would make a respectable paragraph."
 "She had the job she had planned for the last three years."                                                                                                      

In [77]:
sp.EncodeAsPieces(sampleforsp[1])

15-element Array{String,1}:
 "▁"       
 "G"       
 "od"      
 "▁is"     
 "▁"       
 "G"       
 "re"      
 "at"      
 "!"       
 "▁"       
 "I"       
 "▁won"    
 "▁a"      
 "▁lottery"
 "."       

In [78]:
E1 = sp.EncodeAsIds(sampleforsp[1])

15-element Array{Int64,1}:
    13
     1
  5648
    25
    13
     1
    99
   721
   187
    13
     1
   230
    21
 17518
     9

In [79]:
E2 = sp.EncodeAsIds(sampleforsp[2])

32-element Array{Int64,1}:
    13
     1
   410
    65
    66
 13527
    19
    14
   132
   818
    24
    41
    74
     ⋮
    15
    32
    23
 22568
    30
    59
    83
   233
    21
 22739
 20599
     9

In [80]:
E3 = sp.EncodeAsIds(sampleforsp[3])

15-element Array{Int64,1}:
   13
    1
  438
   41
   14
 1205
   39
   41
 2035
   26
   14
  236
  132
  122
    9

if we are going to feed all the three sentence in the model we need to add pad token for all the input we for now we can to do it manually later will be automatically can be provided


In [81]:
E = convertVector[E1,E2,E3]


UndefVarError: UndefVarError: convertVector not defined

In [82]:
indices

30×3 Array{Int64,2}:
  5475    822  1080
    26     66    42
   375     67    15
   188  13528  1206
    32     20    40
   231     15    42
    22    133  2036
 17519    819    27
    10     25    15
     2     42   237
     2     75   133
     2    881   123
     2     21    10
     ⋮             
     2     16     2
     2     33     2
     2     24     2
     2  22569     2
     2     31     2
     2     60     2
     2     84     2
     2    234     2
     2     22     2
     2  22740     2
     2  20600     2
     2     10     2

In [83]:
seg_idx = ones(Int, size(indices)...)

30×3 Array{Int64,2}:
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 ⋮      
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1

Now its time to use released weight 

In [84]:
using Transformers.Basic
using Flux
using Flux: loadparams!

In [85]:
using BSON: @save, @load
@load "/home/iamtejas/Downloads/albert_base_v1.bson.tfbson" config weights vocab

In [123]:
tok_emb = Embed(
    config["embedding_size"],
    config["vocab_size"]
  )

  seg_emb = Embed(
    config["embedding_size"],
    config["type_vocab_size"]
  )

  posi_emb = PositionEmbedding(
    config["embedding_size"],
    config["max_position_embeddings"];
    trainable = true
  )

 emb_post = Positionwise(
    LayerNorm(
      config["embedding_size"]
    ),
       # Dropout(
       #     config["hidden_dropout_prob"]
       # ) there is some problem in loading dropout
  )

embedding = Dict{Symbol, Any}()

Dict{Symbol,Any} with 0 entries

In [124]:
vnames = keys(weights)
embeddings_weights = filter(name->occursin("embeddings", name), vnames)

Set(["bert/embeddings/token_type_embeddings", "bert/embeddings/LayerNorm/gamma", "bert/embeddings/word_embeddings", "bert/embeddings/LayerNorm/beta", "bert/embeddings/position_embeddings"])

In [125]:
 for k ∈ embeddings_weights
        if occursin("LayerNorm/gamma", k)
            loadparams!(emb_post[1].diag.α', [weights[k]]) #there is some problem with loading
            embedding[:postprocessor] = emb_post
        elseif occursin("LayerNorm/beta", k)
            loadparams!(emb_post[1].diag.β', [weights[k]])
        elseif occursin("word_embeddings", k)
            loadparams!(tok_emb.embedding, [weights[k]])
            embedding[:tok] = tok_emb
        elseif occursin("position_embeddings", k)
            loadparams!(posi_emb.embedding, [weights[k]])
            embedding[:pe] = posi_emb
        elseif occursin("token_type_embeddings", k)
             loadparams!(seg_emb.embedding, [weights[k]])
            embedding[:segment] = seg_emb
        else
            @warn "unknown variable: $k"
        end
    end

In [126]:
 albertembed = CompositeEmbedding(;embedding...)

CompositeEmbedding(tok = Embed(128), segment = Embed(128), pe = PositionEmbedding(128, max_len=512), postprocessor = Positionwise(LayerNorm(128)))

In [127]:
embedding_wordpiece = albertembed(tok=indices, segment=seg_indices)

128×30×3 Array{Float32,3}:
[:, :, 1] =
 -0.591439   1.13699    -0.025718   …   0.831079   0.704506    0.440932 
 -0.799194   0.0305692  -0.438631      -0.12549    0.0807379   0.121085 
  2.17368   -0.469415    0.284542      -0.720768  -0.75624    -0.661375 
  2.52347    1.03309     0.92254        0.410215   0.167004    0.0673373
 -0.176611  -1.4789     -0.681304       1.38727    1.40006     1.2112   
  1.16329    1.35542     0.611702   …   1.28013    1.36642     1.50305  
 -1.98747   -1.66523     0.601882       2.24999    2.35954     2.23884  
 -0.310094   2.92642     0.415302       2.34161    2.22562     2.20565  
 -1.37891   -1.38538     0.118553       0.368      0.174424    0.377196 
  1.39736   -0.675551   -0.398878       1.09734    1.24373     0.998397 
 -1.81548   -0.950838   -0.368885   …  -1.78001   -2.07668    -2.44817  
  0.018042  -1.26416    -0.397725       1.02105    1.06319     1.03057  
 -1.21238    0.840682    0.598232      -1.14324   -1.12482    -1.15721  
  ⋮         

here we have embedding for or input 